In [40]:
from ultralytics import YOLO
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import supervision as sv
import numpy as np

In [41]:
# Define input output paths
video_path = '../data/inputs/lebron_1.mp4'
output_dir = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput.mp4'
output_dir_aug = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput_aug.mp4'

model_aug_path = '/Users/treyshanks/data_science/Court_Vision/notebooks/lebron_aug_best.pt'
first_model_path = '/Users/treyshanks/data_science/Court_Vision/lebron_collab/weights/best.pt'


In [42]:
# # Load a pre-trained YOLOv8 model
# model = YOLO(model=first_model_path)

# results = model.predict(
#     source=video_path,       # Input video path
#     # classes=classes_to_detect, # which classes of objects are we looking for
#     vid_stride=5,            # Skip frames to speed up inference
#     conf=0.75,                # Confidence threshold
#     iou=0.5,                 # IoU threshold for non-maximum suppression
#     max_det=4,             # Maximum number of detections per image
#     imgsz=640,               # resize video frame
#     device='mps',           # Use GPU for inference
#     half=True,               # Use FP16 precision for faster inference
#     project=output_dir,      # Output directory for the video
#     stream=True,            # for larger than memory operations
#     # visualize=True
# )

# # Process results list
# for result in results:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     obb = result.obb  # Oriented boxes object for OBB outputs
#     result.show()  # display to screen

In [43]:
# model = YOLO(model_aug_path)

# # Function to run inference on a video
# def run_inference_on_video(video_path, output_path):
#     # Capture the video
#     cap = cv2.VideoCapture(video_path)
#     if not cap.isOpened():
#         print("Error: Could not open video.")
#         return

#     # Get video properties
#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = cap.get(cv2.CAP_PROP_FPS)

#     # Define the codec and create VideoWriter object
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 files
#     out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Run inference on the frame
#         results = model.predict(
#             source=frame,
#             device='mps', 
#             conf=0.5,                
#             iou=0.5,                 
#             imgsz=640                
#         )

#         # Iterate over the results and process detections
#         for result in results:
#             boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
#             confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
#             class_ids = result.boxes.cls.cpu().numpy().astype(int)  # Class IDs

#             for box, conf, class_id in zip(boxes, confidences, class_ids):
#                 x1, y1, x2, y2 = map(int, box)
#                 label = model.model.names[class_id]  # Get class label
#                 text = f'{label} {conf:.2f}'
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                 cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#             # Write the annotated frame to the output video
#             out.write(frame)

#             # Optionally display the frame
#             # cv2.imshow('YOLOv8 Inference', frame)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 cap.release()
#                 out.release()
#                 cv2.destroyAllWindows()
#                 return

#     # Release resources
#     cap.release()
#     out.release()
#     cv2.destroyAllWindows()

# run_inference_on_video(video_path, output_dir_aug)


TODO: Fine-tuning --> 
identify type of video - camera angle, single game or continuous video
log findings from inference?
does the model track boxes frame to frame -- player tracking?
identify teams? identify a specific player and location? 
metric -- player movement? 
fixed camera angel vs jump cuts?
Need to identify what kind of video to run inference on

In [44]:
# Define input output paths
video_path = '../data/inputs/lebron_short_1.mp4'
output_dir = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput.mp4'
output_dir_aug = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput_aug.mp4'

model_aug_path = '/Users/treyshanks/data_science/Court_Vision/notebooks/lebron_aug_best.pt'
first_model_path = '/Users/treyshanks/data_science/Court_Vision/lebron_collab/weights/best.pt'

In [47]:
device = 'mps'
model = YOLO(model_aug_path)
model.model.to(device)

tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    tracked_detections = tracker.update_with_detections(detections)

    # generate tracking labels
    labels = [
        f"{model.model.names[class_id]} {confidence:.2f}"
        for class_id, tracker_id, confidence in zip(tracked_detections.class_id, tracked_detections.tracker_id, tracked_detections.confidence)
    ]

    if len(labels) != len(detections):
        print(f"The number of labels provided ({len(labels)}) does not match the number of detections ({len(detections)}).")
        return frame

    # Annotate the frame with the filtered detections
    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
    
    return annotated_frame

# Process the video
sv.process_video(
    source_path=video_path,
    target_path=output_dir_aug,
    callback=callback
)


0: 384x640 1 LeBronJames, 99.4ms
Speed: 1.5ms preprocess, 99.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 LeBronJamess, 93.1ms
Speed: 1.3ms preprocess, 93.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
The number of labels provided (1) does not match the number of detections (2).

0: 384x640 2 LeBronJamess, 91.8ms
Speed: 1.5ms preprocess, 91.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
The number of labels provided (1) does not match the number of detections (2).

0: 384x640 1 LeBronJames, 87.5ms
Speed: 1.2ms preprocess, 87.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 LeBronJames, 99.3ms
Speed: 1.4ms preprocess, 99.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 LeBronJames, 99.2ms
Speed: 1.3ms preprocess, 99.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 LeBronJames, 99.0ms
Speed: 1.3ms preproc